In [ ]:
pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 13.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 시험지 추출

In [ ]:
import fitz  # PyMuPDF
import re
import pandas as pd

In [ ]:
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):  # 전체 페이지 반복
        page = document.load_page(page_num)
        text += page.get_text() + "\n"
    return text

In [ ]:
print(extract_text_from_pdf(pdf_path))

홀수형
1
1
20
[1～3] 다음 글을 읽고 물음에 답하시오.
독서는 독자가 목표한 결과에 도달하기 위해 글을 읽고 의미를 
구성하는 인지 행위이다. 성공적인 독서를 위해서는 초인지가 
중요하다. 독서에서의 초인지는 독자가 자신의 독서 행위에 대해 
인지하는 것으로서 자신의 독서 과정을 점검하고 조정하는 
역할을 한다.
초인지는 글을 읽기 시작한 후 지속적으로 이루어지는 
점검 과정에 동원된다. 독자는 가장 적절하다고 판단한 독서 
전략을 사용하여 독서를 진행하는데, 그 전략이 효과적이고 
문제가 없는지를 평가하며 점검한다. 효과적이지 않거나 
문제가 있다고 판단하면 이를 해결해야 한다. 문제가 무엇
인지 분명하지 않은 경우에는 독서 중에 떠오르는 생각들을 
살펴보고 그중 독서의 진행을 방해하는 생각들을 분류해 
보는 방법으로 문제점이 무엇인지 파악할 수 있다. 독서가 
중단 없이 이어지는 상태이지만 문제가 발생한 것을 독자 
자신이 인지하지 못하는 경우도 있다. 의도한 목표에 부합하지 
않는 방법으로 읽기를 진행하거나 자신이 이해한 정도를 
판단하지 못하는 예가 그것이다. 문제 발생 여부의 점검을 
위해서는 독서 진행 중간중간에 이해한 내용을 정리하는 
방법을 사용할 수 있다.
초인지는 문제를 해결하기 위해 독서 전략을 조정하는 과정
에도 동원된다. 독서 목표를 고려하여, 독자는 ㉠지금 사용하고 
있는 전략을 계속 사용할 것인지를 판단해야 한다. 또 ㉡문제 
해결을 위한 다른 전략에는 무엇이 있는지, ㉢각 전략의 특징과 
사용 절차, 조건 등은 무엇인지 알아야 한다. 또한 독자 자신이 
사용할 수 있는 전략이 무엇인지, ㉣전략들의 적절한 적용 
순서가 무엇인지, ㉤현재의 상황에서 최적의 전략이 무엇인지 
판단하여 새로운 전략을 선택한다. 선택한 전략을 수행하는 
과정에서 독자는 초인지를 활용하여 점검과 조정을 되풀이하며 
능동적으로 의미를 구성해 간다.
1. 윗글을 이해한 내용으로 적절하지 않은 것은?
①독서 전략을 선택할 때 독서의 목표를 고려할 필요가 

In [ ]:
def split_text_into_components(text):
    questions = []
    choices = []
    passages = []

    current_question = None
    current_choices = []
    current_passage = None

    passage_pattern = re.compile(r'^\[\d+～\d+\]|\[다음 글을 읽고 물음에 답하시오\]')
    question_pattern = re.compile(r'^\d+\.\s')
    choice_pattern = re.compile(r'^[①②③④⑤]')
    unwanted_text_pattern = re.compile(r'(이 문제지에 관한 저작권은 한국교육과정평가원에 있습니다.|홀수형|짝수형|\d+\s?페이지)')

    # 불필요한 텍스트 제거
    text = unwanted_text_pattern.sub('', text)

    lines = text.split('\n')

    for line in lines:
        line = line.strip()
        if not line:
            continue

        if passage_pattern.search(line):
            if current_question or current_choices:
                questions.append(current_question)
                choices.append(current_choices)
                passages.append(current_passage)
            current_passage = line
            current_question = None
            current_choices = []
        elif question_pattern.search(line):
            if current_question:
                questions.append(current_question)
                choices.append(current_choices)
                passages.append(current_passage)
            current_question = line
            current_choices = []
        elif choice_pattern.search(line):
            current_choices.append(line)
        else:
            if current_passage:
                current_passage += " " + line

    if current_question or current_choices:
        questions.append(current_question)
        choices.append(current_choices)
        passages.append(current_passage)

    return passages, questions, choices


In [ ]:
def filter_passages(questions, choices, passages):
    example_pattern = re.compile(r'<보기>')
    filtered_questions = []
    filtered_choices = []
    filtered_passages = []

    for i in range(len(questions)):
        if questions[i] and not example_pattern.search(questions[i]):
            filtered_questions.append(questions[i])
            filtered_choices.append(choices[i])
            filtered_passages.append(passages[i])

    return filtered_questions, filtered_choices, filtered_passages


In [ ]:
def print_questions(questions, choices, passages):
    for i in range(len(questions)):
        print(f"질문: {questions[i]}")
        print("선지: " + " ".join(choices[i]))
        print(f"지문: {passages[i]}")
        print("\n")

text = extract_text_from_pdf(pdf_path)

passages, questions, choices = split_text_into_components(text)

filtered_questions, filtered_choices, filtered_passages = filter_passages(questions, choices, passages)

print_questions(filtered_questions, filtered_choices, filtered_passages)

질문: 1. 윗글을 이해한 내용으로 적절하지 않은 것은?
선지: ①독서 전략을 선택할 때 독서의 목표를 고려할 필요가 있다. ②독서 전략의 선택을 위해 개별 전략들에 대한 지식이 필요하다. ③독서 목표의 달성을 위해 독자는 자신의 독서 행위에 대해 ④독서 문제의 해결을 위해 독자는 자신이 사용할 수 있는 ⑤독서 문제를 해결하기 위해 새로 선택한 전략은 점검과
지문: [1～3] 다음 글을 읽고 물음에 답하시오. 독서는 독자가 목표한 결과에 도달하기 위해 글을 읽고 의미를 구성하는 인지 행위이다. 성공적인 독서를 위해서는 초인지가 중요하다. 독서에서의 초인지는 독자가 자신의 독서 행위에 대해 인지하는 것으로서 자신의 독서 과정을 점검하고 조정하는 역할을 한다. 초인지는 글을 읽기 시작한 후 지속적으로 이루어지는 점검 과정에 동원된다. 독자는 가장 적절하다고 판단한 독서 전략을 사용하여 독서를 진행하는데, 그 전략이 효과적이고 문제가 없는지를 평가하며 점검한다. 효과적이지 않거나 문제가 있다고 판단하면 이를 해결해야 한다. 문제가 무엇 인지 분명하지 않은 경우에는 독서 중에 떠오르는 생각들을 살펴보고 그중 독서의 진행을 방해하는 생각들을 분류해 보는 방법으로 문제점이 무엇인지 파악할 수 있다. 독서가 중단 없이 이어지는 상태이지만 문제가 발생한 것을 독자 자신이 인지하지 못하는 경우도 있다. 의도한 목표에 부합하지 않는 방법으로 읽기를 진행하거나 자신이 이해한 정도를 판단하지 못하는 예가 그것이다. 문제 발생 여부의 점검을 위해서는 독서 진행 중간중간에 이해한 내용을 정리하는 방법을 사용할 수 있다. 초인지는 문제를 해결하기 위해 독서 전략을 조정하는 과정 에도 동원된다. 독서 목표를 고려하여, 독자는 ㉠지금 사용하고 있는 전략을 계속 사용할 것인지를 판단해야 한다. 또 ㉡문제 해결을 위한 다른 전략에는 무엇이 있는지, ㉢각 전략의 특징과 사용 절차, 조건 등은 무엇인지 알아야 한다. 또한 독자 자신이 사용할 수 있는 전략이 무엇인지, ㉣전략들의 적절한 적용 순서가

In [ ]:
import os
import pandas as pd

def save_to_csv(questions, choices, passages, output_path):
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    # Define column names and save to CSV
    data = {'질문': questions, '선지': choices, '지문': passages}
    df = pd.DataFrame(data)
    df.to_csv(output_path, index=False, encoding='utf-8-sig')

In [ ]:
# 경로와 파일명을 지정합니다.
pdf_path = '/content/drive/MyDrive/빅프/수능/2024_국어영역_홀수형.pdf'
output_csv_path = '/content/drive/MyDrive/빅프/2024_수능_국어.csv'

In [ ]:
# PDF에서 전체 텍스트 추출
pdf_text = extract_text_from_pdf(pdf_path)

# 텍스트를 지문, 질문, 선지로 분리
passages, questions, choices = split_text_into_components(pdf_text)

# <보기>가 포함된 문제를 제외
filtered_questions, filtered_choices, filtered_passages = filter_passages(questions, choices, passages)

# CSV 파일로 저장
save_to_csv(filtered_questions, filtered_choices, filtered_passages, output_csv_path)

### 답안 추출

In [ ]:
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):  # 전체 페이지 반복
        page = document.load_page(page_num)
        text += page.get_text() + "\n"
    return text

In [ ]:
pdf_path = '/content/drive/MyDrive/빅프/수능/해설/2024_국어영역_해설지(화법과 작문).pdf'

In [ ]:
# PDF에서 전체 텍스트 추출
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text)

2024학년도 대학수학능력시험  
국어
국어
국어영역
영역
영역 
 
 정답 및 해설
정답 및 해설
정답 및 해설
1
[1~3] 독서
[1~3] 독서 이론, ‘읽기 과정에서 초인지의 역할’
지문해설 : 이 글은 독서에 초인지가 어떻게 동원되는지에 대해 설명하고 있다. 독서
에서의 초인지는 독자가 자신의 독서 행위에 대해 인지하는 것으로 자신의 독서 과정
을 점검하고 조정하는 역할을 한다. 초인지는 글을 읽기 시작한 후 지속적으로 이루
어지는 점검 과정에 동원된다. 이를 통해 독자는 독서 전략이 효과적이고 문제가 없
는지를 평가하며 문제를 해결한다. 문제가 무엇인지 분명하지 않은 경우나 문제가 발
생한 것을 독자 자신이 인지하지 못하는 경우에도 특정 방법들을 사용하여 문제 발생 
여부를 점검할 수 있다. 또한 초인지는 문제를 해결하기 위해 독서 전략을 조정하는 
과정에도 동원되는데, 독자는 이러한 초인지를 활용하여 점검과 조정을 되풀이하며 
능동적으로 의미를 구성해 간다.
  [주제] 독서 과정을 점검하고 조정하는 역할을 하는 초인지
1. 세부 내용 파악
정답해설 : 3문단에 따르면, 초인지는 문제를 해결하기 위해 독서 전략을 점검하고 
조정하는 과정에 동원된다. 이때 현재의 상황에서 최적의 전략이 무엇인지 판단하여 
새로운 전략을 선택할 수 있다. 따라서 독서 문제를 해결하기 위해 새로 선택한 전략
을 점검과 조정의 과정에서 제외할 필요가 있다고 이해하는 것은 적절하지 않다.
정답 ⑤
[오답피하기] ① 3문단에서 독서 목표를 고려하여 독자가 지금 사용하고 있는 전략을 
계속 사용할 것인지를 판단해야 한다고 제시하고 있다. ② 3문단에서 독서 전략의 점
검과 조정을 위해 각 전략의 특징과 사용 절차, 조건 등이 무엇인지 알아야 한다고 
제시하고 있다. 이를 통해 독서 전략의 선택을 위해 개별 전략들에 대한 지식이 필요
하다고 이해할 수 있다. ③ 1문단에서 독서는 독자가 목표한 결과에 도달하기 위해 
글을 읽고 의미를 구성하는 인지 행위라고 제시하고 있다. ④ 

답만 추출

In [ ]:
import fitz

In [ ]:
def extract_answers(text):
    answers = []

    answer_pattern = re.compile(r'정답\s([①②③④⑤])')

    # 텍스트를 줄 단위로 분리
    lines = text.split('\n')

    for line in lines:
        line = line.strip()
        if not line:
            continue
        answer_match = answer_pattern.search(line)
        if answer_match:
            # 정답을 감지
            answers.append(answer_match.group(1))

    return answers

def print_answers(answers):
    for answer in answers:
        print(answer)

In [ ]:
# 정답만 추출
answers = extract_answers(pdf_text)

# 정답 출력
print_answers(answers)

⑤
③
①
⑤
③
②
②
③
①
⑤
②
③
①
④
④
⑤
④
②
①
③
⑤
②
①
③
③
②
⑤
①
④
①
②
②
③
④
④
⑤
④
④
⑤
①
③
③
②
⑤
①


In [ ]:
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):  # 전체 페이지 반복
        page = document.load_page(page_num)
        text += page.get_text() + "\n"
    return text

def extract_explanations(text):
    explanations = []
    current_explanation = None
    in_explanation = False

    # 패턴 정의: '정답해설' 또는 '[오답피하기]'로 시작하는 텍스트
    explanation_start_pattern = re.compile(r'^(정답해설|정답 해설|정답[ ]해설|\[오답피하기\])')
    explanation_end_pattern = re.compile(r'^정답\s[①②③④⑤⑥⑦⑧⑨⑩]')
    unwanted_text_pattern = re.compile(r'(이 문제지에 관한 저작권은 한국교육과정평가원에 있습니다.|홀수형|짝수형|\d+\s?페이지)')

    # 불필요한 텍스트 제거
    text = unwanted_text_pattern.sub('', text)

    # 텍스트를 줄 단위로 분리
    lines = text.split('\n')

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if explanation_start_pattern.match(line):
            if current_explanation:
                explanations.append(current_explanation.strip())
            current_explanation = line
            in_explanation = True
        elif explanation_end_pattern.match(line):
            if current_explanation:
                explanations.append(current_explanation.strip())
            current_explanation = None
            in_explanation = False
        elif in_explanation:
            current_explanation += " " + line

    if current_explanation:
        explanations.append(current_explanation.strip())

    return explanations

def print_explanations(explanations):
    for explanation in explanations:
        print(explanation)

In [ ]:
# '정답해설'과 '[오답피하기]'로 시작하는 텍스트만 추출
explanations = extract_explanations(pdf_text)

# 추출된 텍스트 출력
print_explanations(explanations)

정답해설 : 3문단에 따르면, 초인지는 문제를 해결하기 위해 독서 전략을 점검하고 조정하는 과정에 동원된다. 이때 현재의 상황에서 최적의 전략이 무엇인지 판단하여 새로운 전략을 선택할 수 있다. 따라서 독서 문제를 해결하기 위해 새로 선택한 전략 을 점검과 조정의 과정에서 제외할 필요가 있다고 이해하는 것은 적절하지 않다.
[오답피하기] ① 3문단에서 독서 목표를 고려하여 독자가 지금 사용하고 있는 전략을 계속 사용할 것인지를 판단해야 한다고 제시하고 있다. ② 3문단에서 독서 전략의 점 검과 조정을 위해 각 전략의 특징과 사용 절차, 조건 등이 무엇인지 알아야 한다고 제시하고 있다. 이를 통해 독서 전략의 선택을 위해 개별 전략들에 대한 지식이 필요 하다고 이해할 수 있다. ③ 1문단에서 독서는 독자가 목표한 결과에 도달하기 위해 글을 읽고 의미를 구성하는 인지 행위라고 제시하고 있다. ④ 3문단에서 독서 문제의 해결을 위해 초인지가 동원되며, 독자 자신이 사용할 수 있는 전략이 무엇인지 판단 하여 새로운 전략을 선택해야 한다고 제시하고 있다. 이는 독서 문제의 해결을 위해 ■ [공통: 독서·문학] 01. ⑤ 02. ③ 03. ① 04. ⑤ 05. ③ 06. ② 07. ② 08. ③ 09. ① 10. ⑤ 11. ② 12. ③ 13. ① 14. ④ 15. ④ 16. ⑤ 17. ④ 18. ② 19. ① 20. ③ 21. ⑤ 22. ② 23. ① 24. ③ 25. ③ 26. ② 27. ⑤ 28. ① 29. ④ 30. ① 31. ② 32. ② 33. ③ 34. ④ 2 독자가 자신이 사용할 수 있는 전략이 무엇인지 알아야 함을 나타낸다. 2. 세부 내용 추론
정답해설 : [A]에서 독서가 중단 없이 이어지는 상태이지만 문제가 발생한 것을 독자 자신이 인지하지 못하는 경우도 있다고 하였다. 그리고 이러한 경우의 예로, 의도한 목표에 부합하지 않는 방법으로 읽기를 진행하는 것을 들고 있다. 따라서 독서 진행 에 문제가 없어 보이더라도 목표에 부합하지 않는 독서가 이루어지

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/drive/MyDrive/빅프/2024_국어영역.csv')

In [4]:
df = df.drop(columns = ['정답','해설','문제유형'], axis = 1)

In [6]:
output_path = '2024 국어 가공.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')